<a href="https://colab.research.google.com/github/raamav/Text-Classification/blob/master/IMDB_Reviews_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Reviews Using RNN

as covered in deeplearning.ai course

In [0]:
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.2.0-rc2'

In [0]:
import tensorflow_datasets as tfds

In [5]:
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW6H17V/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW6H17V/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW6H17V/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

In [0]:
train_data, test_data = imdb['train'],imdb['test']

**IMP** The train and the test sets are iterables, each having 25,000 values of sentences and labels

In [0]:
# GETTING SENTENCES AND LABELS FOR TRAINING DATA

training_sentences = []
training_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

In [18]:
print(training_sentences[2])
print(training_labels[2])

b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.'
0


In [0]:
# GETTING SENTENCES AND LABELS FOR TRAINING DATA

testing_sentences = []
testing_labels = []

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [19]:
print(testing_sentences[3])
print(testing_labels[3])

b'Poor Shirley MacLaine tries hard to lend some gravitas to this mawkish, gag-inducing "feel-good" movie, but she\'s trampled by the run-away sentimentality of a film that\'s not the least bit grounded in reality.<br /><br />This was directed by Curtis Hanson? Did he have a lobotomy since we last heard from him? Hanson can do effective drama sprinkled with comedy, as evidenced by "Wonder Boys." So I don\'t know what happened to him here. This is the kind of movie that doesn\'t want to accept that life is messy and fussy, and that neat, tidy endings (however implausible they might be) might make for a nice closing shot, but come across as utterly phony if the people watching the film have been through anything remotely like what the characters in the film go through.<br /><br />My wife and I made a game of calling out the plot points before they occurred -- e.g. "the old man\'s going to teach her to read and then drop dead." Bingo! This is one of those movies where the characters give l

In [21]:
print("type(training_sentences) :", type(training_sentences))
print(" \n type(training_labels) :", type(training_labels))

print(" \n shape of training_sentences :", len(training_sentences))
print(" \n shape of training_labels :", len(training_labels))

type(training_sentences) : <class 'list'>
 
 type(training_labels) : <class 'list'>
 
 shape of training_sentences : 25000
 
 shape of training_labels : 25000


**IMP** We have successfuly converted the predictors and the labels into lists

In [0]:
# COVERTING THE LABELS TO NUMPY ARRAYS

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [23]:
print(training_labels_final.shape)

(25000,)


In [0]:
# TOKENIZING THE CODE 
# THREE THINGS: ENCODE SEQUENCE AND PAD SEQUENCE. GET WORD_INDEX

vocab_index = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_index, oov_token = oov_tok)

tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

In [26]:
print("length of the unfiltered vocabulary : ", len(word_index))

length of the unfiltered vocabulary :  86539


In [0]:
training_sequence = tokenizer.texts_to_sequences(training_sentences)

training_padded = pad_sequences(training_sequence, maxlen = max_length,padding = "post", truncating = trunc_type)


In [33]:
print("\n shape of training sequence", len(training_sequence))
print("\n shape of padding sequence", training_padded.shape)


 shape of training sequence 25000

 shape of padding sequence (25000, 120)


In [0]:
# IMPLEMENTING SEQUENCE ENCODING AND PADDING ON THE TESTING SET

testing_sequence = tokenizer.texts_to_sequences(testing_sentences)

testing_padded = pad_sequences(testing_sequence, maxlen = max_length, padding = "post", truncating = trunc_type)

In [38]:
print("shape of testing padding sequence", testing_padded.shape)

shape of testing padding sequence (25000, 120)


**We now have the encoded and padded sentences (as numpy arrays) for both the training and test datasets.** 
**Additionally, the labels have been converted to numpy arrays**

### Dense Model with `Flatten` Layer()

**Summary**: 

In [0]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [0]:
# CREATE A DENSE NEURAL NETWORK IN KERAS

vocab_size = vocab_index + 1   # NUMBER OF UNIQUE WORDS IN VOCAB = 10,000 + 1 
embedding_dimensions = 16
sentence_length = max_length.  # LENGTH OF EACH PADDED SENTENCE = 120

dense_model = keras.models.Sequential([
                                          
                  keras.layers.Embedding(input_dim = vocab_size,
                                         output_dim = embedding_dimensions,
                                         input_length = sentence_length),
                                          
                  keras.layers.Flatten(),

                  keras.layers.Dense(6, activation = "relu"),
                  keras.layers.Dense(1, activation = "sigmoid")


])


dense_model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

In [47]:
dense_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160016    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,549
Trainable params: 171,549
Non-trainable params: 0
_________________________________________________________________


In [46]:
history = dense_model.fit(training_padded, training_labels_final, epochs = 10,
                          validation_data = (testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2379 - accuracy: 0.9076 - val_loss: 0.4233 - val_accuracy: 0.8194
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0901 - accuracy: 0.9750 - val_loss: 0.5236 - val_accuracy: 0.8022
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0189 - accuracy: 0.9978 - val_loss: 0.6109 - val_accuracy: 0.8049
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0046 - accuracy: 0.9998 - val_loss: 0.6795 - val_accuracy: 0.8075
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.7315 - val_accuracy: 0.8092
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 8.1086e-04 - accuracy: 1.0000 - val_loss: 0.7861 - val_accuracy: 0.8080
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 4.3599e-04 - accuracy: 1.0000 - val_loss: 0.8266 - val_accuracy: 0.808

### Dense Model with `GlobalAveragePooling1D` layer

**SUMMARY** Slightly less number of parameters

In [49]:
# CLEAR BACKEND DATA

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


# CREATE A DENSE NEURAL NETWORK IN KERAS

vocab_size = vocab_index + 1   # NUMBER OF UNIQUE WORDS IN VOCAB = 10,000 + 1 
embedding_dimensions = 16
sentence_length = max_length.  # LENGTH OF EACH PADDED SENTENCE = 120


dense_model_pooling = keras.models.Sequential([
                                          
                  keras.layers.Embedding(input_dim = vocab_size,
                                         output_dim = embedding_dimensions,
                                         input_length = sentence_length),
                                          
                  keras.layers.GlobalAveragePooling1D(),

                  keras.layers.Dense(6, activation = "relu"),
                  keras.layers.Dense(1, activation = "sigmoid")


])


# VIEW MODEL ARCHITECTURE
dense_model_pooling.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160016    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 102       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 160,125
Trainable params: 160,125
Non-trainable params: 0
_________________________________________________________________


In [50]:
# COMPILE AND FIT MODEL

dense_model_pooling.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

history = dense_model_pooling.fit(training_padded, training_labels_final, epochs = 10,
                          validation_data = (testing_padded, testing_labels_final))


Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5506 - accuracy: 0.7550 - val_loss: 0.4174 - val_accuracy: 0.8147
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3327 - accuracy: 0.8605 - val_loss: 0.3678 - val_accuracy: 0.8359
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2710 - accuracy: 0.8899 - val_loss: 0.3737 - val_accuracy: 0.8353
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2360 - accuracy: 0.9068 - val_loss: 0.3873 - val_accuracy: 0.8346
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2087 - accuracy: 0.9215 - val_loss: 0.4146 - val_accuracy: 0.8289
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1882 - accuracy: 0.9292 - val_loss: 0.4698 - val_accuracy: 0.8120
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1702 - accuracy: 0.9405 - val_loss: 0.4783 - val_accuracy: 0.8175
Epoch 

In [54]:
# A QUICK LOOK AT THE EMBEDDING LAYER

e = dense_model.layers[0]

weights = e.get_weights()[0]

print(weights.shape)

(10001, 16)
